In [ ]:
import pandas as pd
import seaborn as sns
from scipy import stats
import pylab as pl
import numpy as np
from lbhb.psychometric import CachedStanModel
%matplotlib inline

df = pd.read_csv('rate_level_functions_for_bburan.csv')
df.rename(columns=lambda x: x.replace(' ', ''), inplace=True)
df = pd.wide_to_long(df, ['pupil', 'level', 'spikes'], 'cellid', 'idx', sep='_')
df.dropna(inplace=True)
df = df.reset_index().set_index(['cellid', 'pupil', 'level'], verify_integrity=True)['spikes']
df.sort_index(inplace=True)

In [ ]:
sr = df.groupby(['cellid', 'pupil']).min().clip(lower=0.1).unstack()

def sr_dist(sr):
    pl.hist(sr, density=True, bins=100);
    x = np.arange(0, 50, 0.1)
    a, loc, scale = stats.gamma.fit(sr, floc=0)
    y = stats.gamma.pdf(x, a, loc, scale)
    pl.plot(x, y, 'k-')
    y = stats.gamma.pdf(x, 0.5, 0, 1/0.1)
    pl.plot(x, y, 'r-')
    print(f'alpha = {a:.2f}, beta = {1/scale:.2f}')
    
sr_dist(sr[1])
sr_dist(sr[2])

In [ ]:
df.groupby(['pupil', 'level']).mean().unstack('pupil').plot()

In [ ]:
import arviz as az

In [ ]:
fit

In [ ]:
c = 'BOL006b-21-1'
d = df.loc[c].unstack('pupil')[2]

data = {
    'n': len(d),
    'x': d.index.values,
    'y': d.values,
}
fit = hs_model.sampling(data, iter=10000)
az.plot_posterior(fit, ['sr', 'slope', 'threshold'])

In [ ]:
trace = fit.to_dataframe(['sr', 'slope', 'intercept'], permuted=True)
slope = trace['slope'].values
intercept = trace['intercept'].values
sr = trace['sr'].values

x = np.arange(-20, 100)[..., np.newaxis]

y = slope * x + intercept
y = np.clip(y, sr, np.inf)
y_mean = y.mean(axis=1)
pl.plot(d, 'ko')
pl.plot(x, y[:, ::20], 'k-', alpha=0.005)
pl.plot(x, y_mean, 'k-')


In [ ]:
y.shape[-1]/20

In [ ]:
y[:, ::20].shape

In [ ]:
pl.imshow(y, aspect='auto')

In [ ]:
bins = np.arange(500)
bin_centers = bins[:-1] + 0.5
z = np.apply_along_axis(lambda x: np.histogram(x, bins=bins), 1, y)[0]

In [ ]:
y.shape

In [ ]:
len(z)
#counts, bins = np.histogram(y[0], bins=bins)
#pl.plot(bin_centers, counts)

In [ ]:
az.